This encodes the Hamiltonian of the helium hydride molecule HeH$^+$. The energy $\langle H \rangle$ is measured as follows:
$$
\langle H \rangle = p(00) + p(01) - p(10) - p(11).
$$
The lowest energy state is then found using a loop of quantum circuit executions and classical optimization.

In [10]:
import numpy as np

from quantuminspire.credentials import enable_account, get_token_authentication
from quantuminspire.api import QuantumInspireAPI

enable_account('6ea0bc8530ed4c97073aff535b5b52fd617cc6ed')
auth = get_token_authentication()
server_url = r'https://api.quantum-inspire.com'
qi = QuantumInspireAPI(server_url, auth)

def estimate_hamiltonian(params, shots=1024, backend='QX single-node simulator'):
    print(f'Running quantum circuit with params {params}')
    
    qasm = f'''version 1.0
qubits 2

.ansatz
{{Rx q[0], {params[0]} | Rx q[1], {params[1]}}}
{{Rz q[0], {params[2]} | Rz q[1], {params[3]}}}

H q[0]
CNOT q[0], q[1]
H q[1]
CNOT q[1], q[0]

{{Rz q[0], {params[4]} | Rz q[1], {params[5]}}}
{{Rx q[0], {params[6]} | Rx q[1], {params[7]}}}
{{Rz q[0], {params[8]} | Rz q[1], {params[9]}}}

.measurement
measure_all
'''
    backend_type = qi.get_backend_type_by_name(backend)
    result = %time qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=shots)
    
    if result.get('histogram', {}):
        hist = result['histogram']
        expval = hist.get('0') + hist.get('1') - hist.get('2') - hist.get('3')
        return expval
    else:
        reason = result.get('raw_text', 'No reason in result structure.')
        raise RuntimeError(f'Result structure does not contain proper histogram data. {reason}')

In [8]:
from scipy.optimize import minimize

def find_minimum(params, backend):
    return minimize(lambda p: estimate_hamiltonian(p, backend=backend), params, method='nelder-mead',
                    options={'xtol': 1e-12, 'disp': True, 'maxiter': 10})

np.random.seed(111)

## Spin-2 back-end

In [10]:
params = np.random.uniform(low=-np.pi, high=np.pi, size=(10,))
find_minimum(params, 'Spin-2')

Running quantum circuit with params [ 0.704786   -2.07929606 -0.40175303  1.69182601 -1.28600904 -2.20437415
 -3.00035718 -0.5012443  -1.64190853 -1.02003623]
Running quantum circuit with params [ 0.7400253  -2.07929606 -0.40175303  1.69182601 -1.28600904 -2.20437415
 -3.00035718 -0.5012443  -1.64190853 -1.02003623]
Running quantum circuit with params [ 0.704786   -2.18326086 -0.40175303  1.69182601 -1.28600904 -2.20437415
 -3.00035718 -0.5012443  -1.64190853 -1.02003623]
Running quantum circuit with params [ 0.704786   -2.07929606 -0.42184068  1.69182601 -1.28600904 -2.20437415
 -3.00035718 -0.5012443  -1.64190853 -1.02003623]
Running quantum circuit with params [ 0.704786   -2.07929606 -0.40175303  1.77641731 -1.28600904 -2.20437415
 -3.00035718 -0.5012443  -1.64190853 -1.02003623]
Running quantum circuit with params [ 0.704786   -2.07929606 -0.40175303  1.69182601 -1.35030949 -2.20437415
 -3.00035718 -0.5012443  -1.64190853 -1.02003623]
Running quantum circuit with params [ 0.704786

## QX single-node simulator

In [12]:
params = np.random.uniform(low=-np.pi, high=np.pi, size=(10,))
%time find_minimum(params, 'QX single-node simulator')

Running quantum circuit with params [ 1.84702124  2.13986221  1.98050687  3.08476031  0.48552579  1.97145568
 -0.49437436 -2.96913204 -0.28816797 -2.47980934]


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.
Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


CPU times: user 232 ms, sys: 25.8 ms, total: 258 ms
Wall time: 27.2 s
Running quantum circuit with params [ 1.9393723   2.13986221  1.98050687  3.08476031  0.48552579  1.97145568
 -0.49437436 -2.96913204 -0.28816797 -2.47980934]
CPU times: user 195 ms, sys: 15.7 ms, total: 211 ms
Wall time: 22.2 s


RuntimeError: Result structure does not contain proper histogram data. Error occurred in simulation:
b"qx-simulator: ../nptl/pthread_mutex_lock.c:81: __pthread_mutex_cond_lock: Assertion `mutex->__data.__owner == 0' failed.\n"